In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from IPython.display import Markdown
from pycaret.regression import setup
import numpy as np

# Analiza danych World Happiness Report 2005-2023

In [ ]:
happy_df = pd.read_csv("world_happiness_report.csv", sep=";")
happy_df.rename(columns={
    'Country name': 'country',
    'Year': 'year',
    'Happiness Score': 'happiness_score',
    'GDP': 'gdp',
    'Social support': 'social_support',
    'Healthy life expectancy at birth': 'life_expectancy',
    'Freedom to make life choices': 'freedom',
    'Generosity': 'generosity',
    'Perceptions of corruption': 'corruption',
    'Positive affect': 'positive_affect',
    'Negative affect': 'negative_affect',
    'Continent': 'continent',
}, inplace=True)

## Ogólne informacje

#### Dane zawierają informacje o poziomie szczęścia w różnych krajach na przestrzeni lat.

In [ ]:
Markdown(f"""
#### Dane zawierają **{len(happy_df)}** wierszy i **{len(happy_df.columns)}** kolumn.
""")

In [ ]:
min_year = happy_df['year'].min()
max_year = happy_df['year'].max()
Markdown(f"""
#### Dane pochodzą z lat **{min_year}** - **{max_year}**.
""")

In [ ]:
amount_of_countries = happy_df['country'].nunique()
Markdown(f"""
#### Dane dotyczą **{amount_of_countries}** krajów.
""")

#### Ogromna większość kolumn zawiera wartości liczbowe

In [ ]:
sample_df = happy_df.sample(5, random_state=42)

(
    sample_df[["country", "year", "happiness_score", "gdp", "social_support", "life_expectancy", "freedom"]]
    .style
    .hide(axis="index")
)

In [ ]:
(
    sample_df[["generosity", "corruption", "positive_affect", "negative_affect", "continent"]]
    .style
    .hide(axis="index")
)

#### Happiness Score jest wartością, która określa poziom szczęścia w danym kraju. 

In [ ]:
sns.displot(happy_df, x="happiness_score", fill=True)

median = happy_df["happiness_score"].median()
plt.axvline(median, color="black", linestyle="--")
plt.text(x=median + 0.1, y=200, s="Mediana", color="black", fontdict={"size": 18})
plt.text(x=median + 0.1, y=180, s=str(median), color="black", fontdict={"size": 18})
plt.show()

#### Czy świat jest coraz szczęśliwszy? 

#### Pomimo początkowego spadku, średni poziom szczęścia na świecie rośnie!

In [ ]:
plt.figure(figsize=(12, 6))
happy_df.sort_values('happiness_score', inplace=True)

sns.boxplot(happy_df, x='year', y='happiness_score')
medians = happy_df.groupby('year')['happiness_score'].median().values
for i, median in enumerate(medians):
    plt.text(
        x=i,
        y=median + 0.2,
        s=str(round(median, 2)),
        horizontalalignment='center',
        size='medium',
        color='w',
        weight='semibold',
    )

plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.title('Poziom szczęścia na świecie w latach 2005-2023')
plt.xlabel('Rok')
plt.ylabel('Wskaźnik szczęścia')
plt.grid(which='major', color='#cccccc', linestyle='--')

#### Pierwszy zarejestrowany wynik zawiera pomiary jedynie dla kilku krajów z ogromną przewagą krajów europejskich.

In [ ]:
(
    happy_df[happy_df["year"] == 2005]
    .groupby("continent", as_index=False)[["happiness_score"]].count()
    .rename(columns={
        "happiness_score": "count",
    })
    .sort_values("count", ascending=True)
    .plot(
        x="continent",
        y="count",
        kind="barh",
        legend=False,
        title="Liczba krajów objętych badaniem w 2005 roku",
        xlabel="Liczba krajów",
        ylabel="Kontynent",
    )
);

#### W miarę upływu lat, liczba krajów w zestawie danych wzrasta, co pozwala na bardziej reprezentatywne wyniki.

In [ ]:
fig = plt.figure(figsize=(12, 6))
(
    happy_df
    .groupby("year", as_index=False)[["country"]].count()
    .rename(columns={
        "country": "count",
    })
    .plot(
        x="year",
        y="count",
        kind="bar",
        legend=False,
        title="Liczba krajów objętych badaniem w latach 2005-2023",
        xlabel="Rok",
        ylabel="Liczba krajów",
        ax=fig.gca(),
    )
);

#### Świat jest coraz szczęśliwszy, co najbardziej napędza to szczęście w 2023 roku? 

In [ ]:
exp = setup(
    data=happy_df[happy_df["year"] == 2023],
    target="happiness_score",
    session_id=42,
    ignore_features=["country", "year", "continent"],
    normalize=True,
    normalize_method="zscore",
    verbose=False,
)

best_model = exp.compare_models(exclude=["knn"])

In [ ]:
exp.plot_model(best_model, plot="error")

#### Okazuje się, że największy wpływ na wzrost poziomu szczęścia na świecie ma tzw. `social support` czyli wsparcie społeczne!

In [ ]:
exp.plot_model(best_model, plot="feature")

In [ ]:
!jupyter nbconvert 15__notebook_to_presentation.ipynb --to slides --no-input --no-prompt